In [31]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [32]:
print('Load training data...')
df_x_train = pd.read_csv('X_train.csv', header=0, index_col = 0)
df_y_train = pd.read_csv('y_train.csv', header=0, index_col = 0)

Load training data...


# Explore the dataset

In [33]:
df_x_train.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x877,x878,x879,x880,x881,x882,x883,x884,x885,x886
id,,,,,,,,,,,,,,,,,,,,,
0.0,7077.537454,9.266811e+05,1.076365e+06,1029.624479,105.877440,180486.413011,1084.604810,296933.955135,104840.111208,10736.380544,...,3.921803e+11,1.040522e+06,-437724.358877,957.470031,9.888772e+05,3830.501971,98.622927,1771.079992,10008.297422,65052.593208
1.0,4406.423818,1.172821e+06,1.052467e+06,1000.548328,106.827932,154325.607476,1034.480237,337446.411077,107789.409140,10502.289317,...,3.646601e+10,8.996629e+05,-486231.980813,1127.268384,8.374596e+05,1981.239448,97.369765,843.078502,10046.004997,65052.623467
2.0,4460.878690,9.383663e+05,1.040878e+06,1096.946907,104.058397,197813.871177,NaN,373983.030405,109480.853246,10531.423706,...,9.147176e+10,9.575649e+05,NaN,976.904261,9.210706e+05,2334.782218,113.751711,1134.021199,10307.683079,NaN
3.0,7152.388016,9.231857e+05,1.009414e+06,1044.520384,101.089811,182569.308398,NaN,339951.730003,106597.409809,10070.560056,...,2.428226e+11,NaN,-497230.891433,912.933479,1.117182e+06,3711.834265,98.080257,1553.015493,10952.963586,65052.576387
4.0,6531.930242,1.133718e+06,1.032579e+06,1032.803345,102.608707,200161.011648,NaN,332425.590129,107815.955426,10889.035954,...,1.273734e+11,1.059684e+06,-485361.519459,1001.804829,9.523764e+05,3153.167755,100.671564,1271.047509,10048.902807,65052.535207


In [34]:
df_y_train.head()

,y
id,
0.0,75.0
1.0,76.0
2.0,74.0
3.0,70.0
4.0,74.0


# Feature Selection

In [35]:
from numpy import isnan
def count_nan(l):
    return len([1 for x in l.values if isnan(x[0])])
print("number of nan's in each column:")
for col in df_x_train:
    print(col, count_nan(df_x_train[[col]]))

number of nan's in each column:
x0 90
x1 72
x2 80
x3 89
x4 79
x5 86
x6 71
x7 90
x8 90
x9 93
x10 85
x11 93
x12 78
x13 100
x14 92
x15 94
x16 77
x17 75
x18 91
x19 82
x20 89
x21 99
x22 96
x23 79
x24 85
x25 86
x26 82
x27 96
x28 87
x29 98
x30 95
x31 88
x32 78
x33 83
x34 91
x35 83
x36 104
x37 80
x38 77
x39 92
x40 92
x41 79
x42 79
x43 91
x44 79
x45 96
x46 90
x47 77
x48 72
x49 85
x50 98
x51 78
x52 82
x53 68
x54 78
x55 83
x56 78
x57 102
x58 69
x59 97
x60 76
x61 97
x62 84
x63 96
x64 97
x65 92
x66 72
x67 93
x68 95
x69 83
x70 76
x71 90
x72 83
x73 102
x74 91
x75 92
x76 100
x77 106
x78 81
x79 103
x80 93
x81 78
x82 85
x83 94
x84 82
x85 93
x86 98
x87 98
x88 86
x89 86
x90 85
x91 74
x92 80
x93 90
x94 66
x95 96
x96 83
x97 104
x98 87
x99 88
x100 76
x101 88
x102 84
x103 78
x104 79
x105 80
x106 81
x107 95
x108 98
x109 78
x110 89
x111 82
x112 85
x113 81
x114 113
x115 98
x116 92
x117 80
x118 89
x119 86
x120 77
x121 78
x122 100
x123 91
x124 90
x125 67
x126 83
x127 84
x128 93
x129 92
x130 83
x131 75
x132 101
x13

In [81]:
from scipy.stats import pearsonr
X = df_x_train.values
y = df_y_train['y'].values
df_x_test = pd.read_csv('X_test.csv', header=0, index_col = 0)
X_test = df_x_test.values
X_total=np.r_[X,X_test]

# Fill missing values
X_filled = KNN(k=5).fit_transform(X_total)
X=X_filled[0:1212,:]
X_test=X_filled[1212:1988,:]

# df_filled = df_x_train.fillna(df_x_train.mean()).copy()
features = df_filled.iloc[:,1:].columns.tolist()
labels = df_y_train["y"].values
correlations = {}
for f in features:
    data_temp = df_filled[[f]]
    x1 = data_temp[f].values
    key = f
    correlations[key] = pearsonr(x1,labels)[0]
print("Correlations of each feature against age")
data_correlations = pd.DataFrame(correlations, index=['correlation']).T
indices_desc = data_correlations['correlation'].abs().sort_values(ascending=False).index
data_correlations.loc[indices_desc][abs(data_correlations['correlation']) >= 0.2]

/Users/Xivid/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


Correlations of each feature against age


/Users/Xivid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,correlation
x746,0.455891
x82,0.451206
x685,-0.450200
x333,0.436991
x391,0.432717
x722,0.432330
x882,-0.429216
x751,-0.415688
x291,-0.409631
x96,0.408898


In [82]:
selected_features = data_correlations['correlation'].abs().sort_values(ascending=False).loc[abs(data_correlations['correlation']) >= 0.1].index
selected_features

Index(['x746', 'x82', 'x685', 'x333', 'x391', 'x722', 'x882', 'x751', 'x291',
       'x96',
       ...
       'x361', 'x499', 'x416', 'x725', 'x15', 'x39', 'x747', 'x624', 'x829',
       'x789'],
      dtype='object', length=203)

# Training LightGBM

In [79]:
X = df_x_train[selected_features].values
X

array([[1.51043839e+03, 7.92533090e+02, 7.57225976e+04, ...,
        2.58807884e+00,            nan, 1.40765036e+06],
       [1.71367954e+03, 2.04384761e+03, 3.45676645e+04, ...,
        2.05220010e+00, 1.40100578e+03, 1.51311443e+06],
       [1.80266925e+03, 1.84017550e+03, 1.02509913e+05, ...,
        2.17282790e+00, 2.23203176e+03, 1.56108941e+06],
       ...,
       [3.64095010e+03, 3.07020894e+03, 5.64951903e+04, ...,
        2.61239274e+00, 2.29902917e+03, 1.63645417e+06],
       [2.08526660e+03, 9.55972468e+02, 1.02771633e+05, ...,
        2.58108680e+00, 2.85602196e+03, 1.63319893e+06],
       [8.51175900e+02, 2.26047823e+02, 1.15412762e+05, ...,
        2.10652134e+00,            nan, 1.40965347e+06]])

In [80]:
y = df_y_train['y'].values
y.shape

(1212,)

In [40]:
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn import svm
from fancyimpute import KNN, IterativeSVD

In [51]:
X = df_x_train[selected_features].values
y = df_y_train['y'].values
df_x_test = pd.read_csv('X_test.csv', header=0, index_col = 0)
X_test = df_x_test[selected_features].values
X_total=np.r_[X,X_test]

# Fill missing values
# imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
# imp = imp.fit(X_total)
# X_total= imp.transform(X_total)
X_filled = KNN(k=3).fit_transform(X_total)
X=X_filled[0:1212,:]
X_test=X_filled[1212:1988,:]

z=np.c_[X,y]
rng = np.random.RandomState(42)
clf = LocalOutlierFactor(n_neighbors=50, contamination=0.08)
clf.fit(z)
y_pred_local = clf.fit_predict(z)

clf = IsolationForest(max_samples=100,random_state=rng,contamination=0.08)
clf.fit(z)
y_pred_iso = clf.predict(z)

num=y_pred_local.size

list_inlier=[]
for i in range(0, num):
    if y_pred_local[i]==1 or y_pred_iso[i]==1:
        list_inlier.append(i)

X=X[list_inlier,:]
y=y[list_inlier]

Imputing row 1/1988 with 14 missing, elapsed time: 5.037
Imputing row 101/1988 with 15 missing, elapsed time: 5.066
Imputing row 201/1988 with 17 missing, elapsed time: 5.096
Imputing row 301/1988 with 15 missing, elapsed time: 5.127
Imputing row 401/1988 with 11 missing, elapsed time: 5.157
Imputing row 501/1988 with 11 missing, elapsed time: 5.186
Imputing row 601/1988 with 20 missing, elapsed time: 5.215
Imputing row 701/1988 with 13 missing, elapsed time: 5.246
Imputing row 801/1988 with 14 missing, elapsed time: 5.279
Imputing row 901/1988 with 14 missing, elapsed time: 5.309
Imputing row 1001/1988 with 19 missing, elapsed time: 5.339
Imputing row 1101/1988 with 14 missing, elapsed time: 5.367
Imputing row 1201/1988 with 15 missing, elapsed time: 5.397
Imputing row 1301/1988 with 16 missing, elapsed time: 5.424
Imputing row 1401/1988 with 14 missing, elapsed time: 5.453
Imputing row 1501/1988 with 16 missing, elapsed time: 5.483
Imputing row 1601/1988 with 12 missing, elapsed time

/Users/Xivid/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/Xivid/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [54]:
len(list_inlier)

1153

In [55]:
X=X_filled[list_inlier, :]
y=df_y_train.loc[list_inlier]

In [56]:
X

array([[1.51043839e+03, 7.92533090e+02, 7.57225976e+04, ...,
        2.58807884e+00, 2.69600634e+03, 1.40765036e+06],
       [1.71367954e+03, 2.04384761e+03, 3.45676645e+04, ...,
        2.05220010e+00, 1.40100578e+03, 1.51311443e+06],
       [1.80266925e+03, 1.84017550e+03, 1.02509913e+05, ...,
        2.17282790e+00, 2.23203176e+03, 1.56108941e+06],
       ...,
       [3.64095010e+03, 3.07020894e+03, 5.64951903e+04, ...,
        2.61239274e+00, 2.29902917e+03, 1.63645417e+06],
       [2.08526660e+03, 9.55972468e+02, 1.02771633e+05, ...,
        2.58108680e+00, 2.85602196e+03, 1.63319893e+06],
       [8.51175900e+02, 2.26047823e+02, 1.15412762e+05, ...,
        2.10652134e+00, 2.13456511e+03, 1.40965347e+06]])

In [60]:
print('Splitting into training and validation dataset')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2) # , random_state = 19960503)

Splitting into training and validation dataset


In [61]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [62]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    #'metric': {'l1', 'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

def custom_r2(preds, train_data):
    labels = train_data.get_label()
    return 'r2', r2_score(labels, preds), True

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                feval=custom_r2,
                valid_sets={lgb_train, lgb_eval},
                early_stopping_rounds=20)

Start training...
[1]	training's l2: 84.6065	training's r2: 0.0577477	valid_1's l2: 89.175	valid_1's r2: 0.0464511
Training until validation scores don't improve for 20 rounds.
[2]	training's l2: 79.5298	training's r2: 0.114287	valid_1's l2: 84.5059	valid_1's r2: 0.0963776
[3]	training's l2: 74.8947	training's r2: 0.165907	valid_1's l2: 80.8849	valid_1's r2: 0.135097
[4]	training's l2: 70.7312	training's r2: 0.212275	valid_1's l2: 77.4085	valid_1's r2: 0.17227
[5]	training's l2: 66.817	training's r2: 0.255868	valid_1's l2: 74.196	valid_1's r2: 0.206622
[6]	training's l2: 63.1752	training's r2: 0.296426	valid_1's l2: 71.0382	valid_1's r2: 0.240387
[7]	training's l2: 59.7107	training's r2: 0.335009	valid_1's l2: 68.2267	valid_1's r2: 0.270452
[8]	training's l2: 56.5925	training's r2: 0.369736	valid_1's l2: 65.9021	valid_1's r2: 0.295308
[9]	training's l2: 53.8323	training's r2: 0.400476	valid_1's l2: 63.7767	valid_1's r2: 0.318035
[10]	training's l2: 51.2168	training's r2: 0.429605	valid

[95]	training's l2: 4.78251	training's r2: 0.946738	valid_1's l2: 31.6343	valid_1's r2: 0.661734
[96]	training's l2: 4.69237	training's r2: 0.947742	valid_1's l2: 31.556	valid_1's r2: 0.662572
[97]	training's l2: 4.59792	training's r2: 0.948794	valid_1's l2: 31.5329	valid_1's r2: 0.662819
[98]	training's l2: 4.51498	training's r2: 0.949717	valid_1's l2: 31.4975	valid_1's r2: 0.663197
[99]	training's l2: 4.43682	training's r2: 0.950588	valid_1's l2: 31.4255	valid_1's r2: 0.663967
[100]	training's l2: 4.34898	training's r2: 0.951566	valid_1's l2: 31.3776	valid_1's r2: 0.664479
[101]	training's l2: 4.26065	training's r2: 0.95255	valid_1's l2: 31.4298	valid_1's r2: 0.663921
[102]	training's l2: 4.18238	training's r2: 0.953421	valid_1's l2: 31.3751	valid_1's r2: 0.664506
[103]	training's l2: 4.09429	training's r2: 0.954402	valid_1's l2: 31.3193	valid_1's r2: 0.665102
[104]	training's l2: 4.02163	training's r2: 0.955212	valid_1's l2: 31.2434	valid_1's r2: 0.665913
[105]	training's l2: 3.9482

[181]	training's l2: 1.13831	training's r2: 0.987323	valid_1's l2: 30.2197	valid_1's r2: 0.676861
[182]	training's l2: 1.11661	training's r2: 0.987564	valid_1's l2: 30.2187	valid_1's r2: 0.676871
[183]	training's l2: 1.09922	training's r2: 0.987758	valid_1's l2: 30.2229	valid_1's r2: 0.676826
[184]	training's l2: 1.0805	training's r2: 0.987967	valid_1's l2: 30.2265	valid_1's r2: 0.676788
[185]	training's l2: 1.06308	training's r2: 0.988161	valid_1's l2: 30.2312	valid_1's r2: 0.676738
[186]	training's l2: 1.04468	training's r2: 0.988366	valid_1's l2: 30.1936	valid_1's r2: 0.67714
[187]	training's l2: 1.02379	training's r2: 0.988598	valid_1's l2: 30.1918	valid_1's r2: 0.677159
[188]	training's l2: 1.00708	training's r2: 0.988784	valid_1's l2: 30.1508	valid_1's r2: 0.677598
[189]	training's l2: 0.991665	training's r2: 0.988956	valid_1's l2: 30.1569	valid_1's r2: 0.677532
[190]	training's l2: 0.975114	training's r2: 0.98914	valid_1's l2: 30.1231	valid_1's r2: 0.677893
[191]	training's l2: 

[271]	training's l2: 0.385861	training's r2: 0.995703	valid_1's l2: 29.8706	valid_1's r2: 0.680593
[272]	training's l2: 0.380898	training's r2: 0.995758	valid_1's l2: 29.8699	valid_1's r2: 0.680601
[273]	training's l2: 0.376208	training's r2: 0.99581	valid_1's l2: 29.869	valid_1's r2: 0.680611
[274]	training's l2: 0.371475	training's r2: 0.995863	valid_1's l2: 29.8676	valid_1's r2: 0.680625
[275]	training's l2: 0.366888	training's r2: 0.995914	valid_1's l2: 29.8506	valid_1's r2: 0.680808
Early stopping, best iteration is:
[255]	training's l2: 0.462419	training's r2: 0.99485	valid_1's l2: 29.8361	valid_1's r2: 0.680962


# Predict and output submission

In [19]:
print('Load testing data...')
df_x_test = pd.read_csv('X_test.csv', header=0, index_col = 0)
X_test = df_x_test[selected_features].values
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred

Load testing data...


array([73.72145815, 73.39441609, 73.43724884, 73.3840864 , 73.62956222,
       73.20615037, 73.4502431 , 73.6945529 , 75.56887193, 73.26732154,
       73.72145815, 73.50018263, 73.62956222, 73.62956222, 73.62956222,
       72.99224133, 72.7881334 , 72.55383697, 73.73279552, 73.65318182,
       73.82740721, 73.52924457, 73.26585045, 73.49953291, 75.70808134,
       73.39221458, 73.42504478, 72.73593694, 73.46685323, 73.59473995,
       73.1977275 , 73.1934133 , 73.63686998, 73.4405132 , 73.59995782,
       73.62956222, 73.63660057, 73.50690138, 73.6194166 , 73.42242453,
       73.62956222, 73.51855836, 73.61309763, 72.93190697, 73.62956222,
       73.48134443, 73.39447799, 73.62956222, 73.22097132, 73.40828671,
       73.62956222, 75.45467264, 73.66035838, 73.59995782, 72.86541098,
       73.40828671, 73.62956222, 73.63660057, 73.00522527, 73.61309763,
       73.27624203, 73.51132626, 73.73279552, 73.32983774, 73.63671722,
       73.09752871, 75.76768517, 73.41942912, 73.62956222, 72.33

In [20]:
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,x))
f.close()

# Randomly find best result

In [74]:
best = 0
X = df_x_train[selected_features].values
y = df_y_train['y'].values
df_x_test = pd.read_csv('X_test.csv', header=0, index_col = 0)
X_test = df_x_test[selected_features].values
X_total=np.r_[X,X_test]

# Fill missing values
# imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
# imp = imp.fit(X_total)
# X_total= imp.transform(X_total)
X_filled = KNN(k=5).fit_transform(X_total)
X=X_filled[0:1212,:]
X_test=X_filled[1212:1988,:]

z=np.c_[X,y]
clf = LocalOutlierFactor()
clf.fit(z)
y_pred_local = clf.fit_predict(z)

rng = np.random.RandomState(42)
clf = IsolationForest(random_state=rng)
clf.fit(z)
y_pred_iso = clf.predict(z)

num=y_pred_local.size

list_inlier=[]
for i in range(0, num):
    if y_pred_local[i]==1 and y_pred_iso[i]==1:
        list_inlier.append(i)

X=X[list_inlier,:]
y=y[list_inlier]

for i in range(100):
    print("Running #{}".format(i))
    print('Splitting into training and validation dataset')
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15) # , random_state = 19960503)
    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    # specify your configurations as a dict
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        #'metric': {'l1', 'l2'},
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }

    def custom_r2(preds, train_data):
        labels = train_data.get_label()
        return 'r2', r2_score(labels, preds), True

    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    feval=custom_r2,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=20)
    y_val_p = gbm.predict(X_val, num_iteration=gbm.best_iteration)
    
    result = r2_score(y_val, y_val_p)
    if result > best or result > 0.70:
        print("New Best {}".format(result))
        best = result
        y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        f = open("submission{0:.4f}.csv".format(best), "w")
        f.write("id,y\n")
        for i,x in enumerate(y_pred):
            f.write("{},{}\n".format(i,x))
        f.close()
    best

Imputing row 1/1988 with 14 missing, elapsed time: 4.923
Imputing row 101/1988 with 15 missing, elapsed time: 4.952
Imputing row 201/1988 with 17 missing, elapsed time: 4.982
Imputing row 301/1988 with 15 missing, elapsed time: 5.012
Imputing row 401/1988 with 11 missing, elapsed time: 5.042
Imputing row 501/1988 with 11 missing, elapsed time: 5.070
Imputing row 601/1988 with 20 missing, elapsed time: 5.098
Imputing row 701/1988 with 13 missing, elapsed time: 5.129
Imputing row 801/1988 with 14 missing, elapsed time: 5.161
Imputing row 901/1988 with 14 missing, elapsed time: 5.192
Imputing row 1001/1988 with 19 missing, elapsed time: 5.220
Imputing row 1101/1988 with 14 missing, elapsed time: 5.248
Imputing row 1201/1988 with 15 missing, elapsed time: 5.277
Imputing row 1301/1988 with 16 missing, elapsed time: 5.304
Imputing row 1401/1988 with 14 missing, elapsed time: 5.331
Imputing row 1501/1988 with 16 missing, elapsed time: 5.361
Imputing row 1601/1988 with 12 missing, elapsed time

/Users/Xivid/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/Xivid/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Running #0
Splitting into training and validation dataset
Start training...
[1]	training's l2: 77.9562	training's r2: 0.0602533	valid_0's l2: 97.7531	valid_0's r2: 0.0311935
Training until validation scores don't improve for 20 rounds.
[2]	training's l2: 73.3023	training's r2: 0.116354	valid_0's l2: 93.4665	valid_0's r2: 0.0736774
[3]	training's l2: 69.055	training's r2: 0.167555	valid_0's l2: 89.4063	valid_0's r2: 0.113917
[4]	training's l2: 65.2493	training's r2: 0.213433	valid_0's l2: 85.6921	valid_0's r2: 0.150727
[5]	training's l2: 61.4908	training's r2: 0.25874	valid_0's l2: 81.8844	valid_0's r2: 0.188464
[6]	training's l2: 58.2039	training's r2: 0.298363	valid_0's l2: 78.8149	valid_0's r2: 0.218885
[7]	training's l2: 55.0012	training's r2: 0.336971	valid_0's l2: 75.7605	valid_0's r2: 0.249157
[8]	training's l2: 52.2702	training's r2: 0.369893	valid_0's l2: 72.8093	valid_0's r2: 0.278406
[9]	training's l2: 49.6627	training's r2: 0.401325	valid_0's l2: 70.4388	valid_0's r2: 0.3018

[96]	training's l2: 3.66858	training's r2: 0.955776	valid_0's l2: 32.3624	valid_0's r2: 0.679265
[97]	training's l2: 3.58628	training's r2: 0.956768	valid_0's l2: 32.3307	valid_0's r2: 0.679579
[98]	training's l2: 3.50294	training's r2: 0.957773	valid_0's l2: 32.2201	valid_0's r2: 0.680675
[99]	training's l2: 3.42925	training's r2: 0.958661	valid_0's l2: 32.1627	valid_0's r2: 0.681244
[100]	training's l2: 3.35599	training's r2: 0.959544	valid_0's l2: 32.1172	valid_0's r2: 0.681695
[101]	training's l2: 3.28806	training's r2: 0.960363	valid_0's l2: 32.1263	valid_0's r2: 0.681604
[102]	training's l2: 3.22422	training's r2: 0.961133	valid_0's l2: 32.1196	valid_0's r2: 0.681671
[103]	training's l2: 3.15373	training's r2: 0.961982	valid_0's l2: 32.1001	valid_0's r2: 0.681864
[104]	training's l2: 3.08267	training's r2: 0.962839	valid_0's l2: 32.0818	valid_0's r2: 0.682045
[105]	training's l2: 3.01503	training's r2: 0.963654	valid_0's l2: 32.0362	valid_0's r2: 0.682498
[106]	training's l2: 2.9

[14]	training's l2: 39.3231	training's r2: 0.540242	valid_0's l2: 50.4599	valid_0's r2: 0.42019
[15]	training's l2: 37.5597	training's r2: 0.560859	valid_0's l2: 48.986	valid_0's r2: 0.437126
[16]	training's l2: 36.0111	training's r2: 0.578965	valid_0's l2: 47.596	valid_0's r2: 0.453098
[17]	training's l2: 34.5192	training's r2: 0.596408	valid_0's l2: 46.1975	valid_0's r2: 0.469168
[18]	training's l2: 33.0606	training's r2: 0.613462	valid_0's l2: 45.1702	valid_0's r2: 0.480971
[19]	training's l2: 31.717	training's r2: 0.629171	valid_0's l2: 44.6367	valid_0's r2: 0.487101
[20]	training's l2: 30.5178	training's r2: 0.643192	valid_0's l2: 43.9987	valid_0's r2: 0.494433
[21]	training's l2: 29.2441	training's r2: 0.658084	valid_0's l2: 42.9023	valid_0's r2: 0.507031
[22]	training's l2: 28.0508	training's r2: 0.672035	valid_0's l2: 41.7583	valid_0's r2: 0.520176
[23]	training's l2: 26.9726	training's r2: 0.684642	valid_0's l2: 41.336	valid_0's r2: 0.525028
[24]	training's l2: 25.8954	trainin

...

[37]	training's l2: 15.8074	training's r2: 0.814262	valid_0's l2: 37.0255	valid_0's r2: 0.585522
[38]	training's l2: 15.2385	training's r2: 0.820947	valid_0's l2: 36.685	valid_0's r2: 0.589334
[39]	training's l2: 14.7605	training's r2: 0.826564	valid_0's l2: 36.409	valid_0's r2: 0.592424
[40]	training's l2: 14.3151	training's r2: 0.831797	valid_0's l2: 36.3701	valid_0's r2: 0.592859
[41]	training's l2: 13.895	training's r2: 0.836733	valid_0's l2: 36.1313	valid_0's r2: 0.595532
[42]	training's l2: 13.5009	training's r2: 0.841364	valid_0's l2: 35.8977	valid_0's r2: 0.598147
[43]	training's l2: 13.0526	training's r2: 0.846631	valid_0's l2: 35.794	valid_0's r2: 0.599308
[44]	training's l2: 12.7014	training's r2: 0.850758	valid_0's l2: 35.719	valid_0's r2: 0.600147
[45]	training's l2: 12.3584	training's r2: 0.854788	valid_0's l2: 35.6623	valid_0's r2: 0.600782
[46]	training's l2: 11.994	training's r2: 0.85907	valid_0's l2: 35.4906	valid_0's r2: 0.602704
[47]	training's l2: 11.6347	training'

[144]	training's l2: 1.3329	training's r2: 0.984338	valid_0's l2: 31.2794	valid_0's r2: 0.649846
[145]	training's l2: 1.3084	training's r2: 0.984626	valid_0's l2: 31.2456	valid_0's r2: 0.650224
[146]	training's l2: 1.28276	training's r2: 0.984927	valid_0's l2: 31.2323	valid_0's r2: 0.650373
[147]	training's l2: 1.25953	training's r2: 0.9852	valid_0's l2: 31.1996	valid_0's r2: 0.650739
[148]	training's l2: 1.23767	training's r2: 0.985457	valid_0's l2: 31.1602	valid_0's r2: 0.651181
[149]	training's l2: 1.21559	training's r2: 0.985717	valid_0's l2: 31.1519	valid_0's r2: 0.651273
[150]	training's l2: 1.19425	training's r2: 0.985967	valid_0's l2: 31.1682	valid_0's r2: 0.651091
[151]	training's l2: 1.17309	training's r2: 0.986216	valid_0's l2: 31.2114	valid_0's r2: 0.650608
[152]	training's l2: 1.14986	training's r2: 0.986489	valid_0's l2: 31.1216	valid_0's r2: 0.651612
[153]	training's l2: 1.12809	training's r2: 0.986745	valid_0's l2: 31.1061	valid_0's r2: 0.651786
[154]	training's l2: 1.1

[232]	training's l2: 0.330945	training's r2: 0.996111	valid_0's l2: 30.4286	valid_0's r2: 0.65937
[233]	training's l2: 0.326245	training's r2: 0.996167	valid_0's l2: 30.4123	valid_0's r2: 0.659553
[234]	training's l2: 0.322381	training's r2: 0.996212	valid_0's l2: 30.4189	valid_0's r2: 0.659479
[235]	training's l2: 0.318551	training's r2: 0.996257	valid_0's l2: 30.427	valid_0's r2: 0.659388
[236]	training's l2: 0.314774	training's r2: 0.996301	valid_0's l2: 30.4252	valid_0's r2: 0.659409
[237]	training's l2: 0.311821	training's r2: 0.996336	valid_0's l2: 30.4199	valid_0's r2: 0.659467
[238]	training's l2: 0.308474	training's r2: 0.996375	valid_0's l2: 30.4096	valid_0's r2: 0.659583
[239]	training's l2: 0.304821	training's r2: 0.996418	valid_0's l2: 30.4083	valid_0's r2: 0.659598
[240]	training's l2: 0.301118	training's r2: 0.996462	valid_0's l2: 30.4122	valid_0's r2: 0.659554
[241]	training's l2: 0.298113	training's r2: 0.996497	valid_0's l2: 30.3901	valid_0's r2: 0.659802
[242]	traini

[332]	training's l2: 0.136665	training's r2: 0.998394	valid_0's l2: 30.0932	valid_0's r2: 0.663125
[333]	training's l2: 0.136111	training's r2: 0.998401	valid_0's l2: 30.0917	valid_0's r2: 0.663142
[334]	training's l2: 0.135506	training's r2: 0.998408	valid_0's l2: 30.0869	valid_0's r2: 0.663196
[335]	training's l2: 0.134972	training's r2: 0.998414	valid_0's l2: 30.0886	valid_0's r2: 0.663177
[336]	training's l2: 0.134344	training's r2: 0.998421	valid_0's l2: 30.0898	valid_0's r2: 0.663163
[337]	training's l2: 0.133587	training's r2: 0.99843	valid_0's l2: 30.0826	valid_0's r2: 0.663243
[338]	training's l2: 0.132797	training's r2: 0.99844	valid_0's l2: 30.0762	valid_0's r2: 0.663315
[339]	training's l2: 0.132233	training's r2: 0.998446	valid_0's l2: 30.0757	valid_0's r2: 0.663321
[340]	training's l2: 0.13131	training's r2: 0.998457	valid_0's l2: 30.0693	valid_0's r2: 0.663393
[341]	training's l2: 0.130557	training's r2: 0.998466	valid_0's l2: 30.0692	valid_0's r2: 0.663394
[342]	trainin

[416]	training's l2: 0.101289	training's r2: 0.99881	valid_0's l2: 29.9679	valid_0's r2: 0.664528
[417]	training's l2: 0.101021	training's r2: 0.998813	valid_0's l2: 29.97	valid_0's r2: 0.664504
[418]	training's l2: 0.100802	training's r2: 0.998816	valid_0's l2: 29.9698	valid_0's r2: 0.664506
[419]	training's l2: 0.100609	training's r2: 0.998818	valid_0's l2: 29.9706	valid_0's r2: 0.664497
[420]	training's l2: 0.100253	training's r2: 0.998822	valid_0's l2: 29.97	valid_0's r2: 0.664504
[421]	training's l2: 0.100069	training's r2: 0.998824	valid_0's l2: 29.9688	valid_0's r2: 0.664518
[422]	training's l2: 0.0998229	training's r2: 0.998827	valid_0's l2: 29.9675	valid_0's r2: 0.664532
[423]	training's l2: 0.0996421	training's r2: 0.998829	valid_0's l2: 29.9675	valid_0's r2: 0.664532
[424]	training's l2: 0.0994165	training's r2: 0.998832	valid_0's l2: 29.9638	valid_0's r2: 0.664573
[425]	training's l2: 0.0992306	training's r2: 0.998834	valid_0's l2: 29.9643	valid_0's r2: 0.664568
[426]	train

In [76]:
best

0.713788546803347